In [1]:
import numba
import numpy as np
from collections import defaultdict
from random import shuffle
from functools import partial
DISCRETIZATION_IDS = 25
RHO_VALUES = np.linspace(start=0., stop=1.0, num=DISCRETIZATION_IDS)

In [3]:
def f1(x):
    sum_ = 0
    for elt in x:
        sum_ += elt
    return sum_

In [2]:
@numba.jit
def f2(x):
    sum_ = 0
    for elt in x:
        sum_ += elt
    return sum_

In [5]:
testarr = np.random.randn(1000000)

In [6]:
%%timeit
f1(testarr)

244 ms ± 6.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
f2(testarr)

1.38 ms ± 8.66 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [2]:
def act_optimally(belief, top_k):
    noise_breaking_ties = np.random.randn(*belief.shape) * 1e-5
    belief += noise_breaking_ties
    if len(belief.shape) <= 1:
        return np.sort(np.argpartition(belief, -top_k)[-top_k:])
    else:
        return np.sort(np.argpartition(belief, -top_k, axis=1)[:, -top_k:], axis=1)

In [3]:
def possible_actions(n_items, assortment_size):
    assert assortment_size >= 1
    if assortment_size == 1:
        return [[i] for i in range(n_items)]
    else:
        prev_lists = possible_actions(n_items, assortment_size - 1)
        return [prev_list + [i] for prev_list in prev_lists for i in range(prev_list[-1] + 1, n_items)]

In [14]:
prefs = np.random.rand(100, 5)
action = np.array([0, 1],dtype=int)
rst = 0.9

In [15]:
posteriors_actions = act_optimally(prefs, 3)

posteriors_actions = [tuple(posteriors_actions[ix, :]) for ix in range(15)]
optimal_actions_information = defaultdict(list)
for ix, act in enumerate(posteriors_actions):
    optimal_actions_information[act].append(ix)

opt_actions = {act: (len(theta_idxs), theta_idxs) for
                        act, theta_idxs in optimal_actions_information.items()}

In [16]:
opt_actions

{(0, 1, 3): (2, [0, 7]),
 (0, 3, 4): (3, [1, 3, 14]),
 (0, 2, 3): (2, [2, 8]),
 (2, 3, 4): (3, [4, 9, 12]),
 (0, 1, 4): (3, [5, 10, 13]),
 (1, 2, 3): (1, [6]),
 (1, 3, 4): (1, [11])}

In [17]:
actions_st = np.array([list(key) for key in opt_actions.keys()])
counts_st = np.array([val[0] for val in opt_actions.values()])
thetas_st = []
for val in opt_actions.values():
    thetas_st += val[1]
thetas_st = np.array(thetas_st)

In [19]:
opt_actions = {act: (len(theta_idxs)/100, theta_idxs) for
                        act, theta_idxs in optimal_actions_information.items()}

In [20]:
actions_set = possible_actions(n_items=5, assortment_size=3)

In [21]:
actions_set = np.array(actions_set, dtype=int)

In [22]:
actions_set.shape

(10, 3)

In [26]:
@numba.jit(nopython=True)
def g_full_numba(action, sampled_preferences, opt_actions, probas, thetas):
    """
    :param action:
    :param sampled_preferences: sampled posterior thetas
    :param opt_actions: dictionary {action_tuple:p_action, theta_indices}
    :return:
    """
    g_a = 0.
    probs = 0.
    M = sampled_preferences.shape[0]
    K = action.shape[0]
    n_opt_actions = opt_actions.shape[0]
    
    probas_given_action = np.zeros((M, K))
    no_pick_given_action = np.zeros(shape=(M,))
    p_no_item_action = 0.
    for i in range(M):
        sum_row = 0.
        for ix in range(K):
            val = sampled_preferences[i, action[ix]]
            probas_given_action[i, ix] = val
            sum_row += val
        probas_given_action[i, :] = probas_given_action[i, :] / (1 + sum_row)
        no_pick_given_action[i] = 1 - (sum_row / (1 + sum_row))
        p_no_item_action += no_pick_given_action[i]
    p_no_item_action = p_no_item_action / M

    probs += p_no_item_action
    theta_start = 0
    for i in range(n_opt_actions):
        action_star = opt_actions[i]
        theta_indices = thetas[theta_start:theta_start+probas[i]]
        theta_start += probas[i]
        p_star = probas[i] / M
        p_no_item_a_star_action = 0.
        for theta_indice in theta_indices:
            p_no_item_a_star_action += no_pick_given_action[theta_indice]
        p_no_item_a_star_action = p_no_item_a_star_action / M
        g_a += p_no_item_a_star_action * np.log(p_no_item_a_star_action / (p_star * p_no_item_action))

    for ix in range(K):
        p_item_action = 0.
        for m in range(M):
            p_item_action += probas_given_action[m, ix]
        p_item_action /= M
        if p_item_action > 1e-8:
            probs += p_item_action
            theta_start = 0
            for j in range(n_opt_actions):
                p_star = probas[j] / M
                p_item_a_star_action = 0.
                theta_indices = thetas[theta_start:theta_start+probas[j]]
                for theta_indice in theta_indices:
                    p_item_a_star_action += probas_given_action[theta_indice, ix]
                theta_start += probas[j]
                p_item_a_star_action /= M
                if p_item_a_star_action:
                    g_a += p_item_a_star_action * np.log(p_item_a_star_action / (p_star * p_item_action))
#     assert probs > 0.999, f"{probs}"
#     assert probs < 1.001, f"{probs}"
    return g_a

@numba.jit(nopython=True)
def numba_expected_reward(pref, action):
    M = pref.shape[0]
    K = len(action)
    result = 0.
    for i in range(M):
        temp_sum = 0.
        for k in range(K):
            temp_sum += pref[i, action[k]]
        result += temp_sum / (1 + temp_sum)
    return result / M

@numba.jit(nopython=True)
def delta_2(action, sampled_preferences, r_star):
    x = r_star - numba_expected_reward(action=action, pref=sampled_preferences)
    return x

@numba.jit(nopython=True)
def information_ratio_numba(rho, d1, d2, g1, g2):
    return (d1 * rho + (1 - rho) * d2) ** 2 / (g1 * rho + (1 - rho) * g2)

@numba.jit(nopython=True)
def optimized_ratio_numba(d1, d2, g1, g2):
    n_rho = len(RHO_VALUES)
    possible_values = np.zeros(n_rho)
    for ix in range(n_rho):
        possible_values[ix] = information_ratio_numba(RHO_VALUES[ix], d1, d2, g1, g2)
    min_ = 1e8
    rho_min = -1
    for ix in range(n_rho):
        rho = RHO_VALUES[ix]
        val = possible_values[ix]
        if val < min_:
            rho_min = rho
            min_ = val
    return min_, rho_min

@numba.jit(nopython=True)
def ids_action_selection_numba(n, k, delta_, g_, actions_set, sampled_preferences, r_star, actions_star, counts_star, thetas_star):
    np.random.shuffle(actions_set)
    min_information_ratio = 1e8
    ids_action = actions_set[0]
    n_actions = actions_set.shape[0]
    for i in range(n_actions):
        action1 = actions_set[i]
        g_a1 = g_(action1, sampled_preferences, actions_star, counts_star, thetas_star)
        delta_1 = delta_(action1, sampled_preferences, r_star)
        for j in range(n_actions):
            action2 = actions_set[j]
            g_a2 = g_(action2, sampled_preferences, actions_star, counts_star, thetas_star)
            delta_2 = delta_(action2, sampled_preferences, r_star)
            if (not g_a1) or (not g_a2):
                if delta_1 < delta_2:
                    value = delta_1
                    action_picked = action1
                else:
                    value = delta_2
                    action_picked = action2
                rho = 1. if delta_1 < delta_2 else 0.
            else:
                value, rho = optimized_ratio_numba(d1=delta_1,
                                             d2=delta_2,
                                             g1=g_a1,
                                             g2=g_a2)
                action_picked = action1 if np.random.rand() <= rho else action2
            if value < min_information_ratio:
#                 print(value)
                min_information_ratio = value
                ids_action = action_picked
#     print(ids_action)
    return ids_action

@numba.jit(nopython=True)
def v_ids_numba(action, sampled_preferences, actions_star, counts, thetas):
    """
    :param action: 1D array of size (K,) with the indices of the current action
    :param sampled_preferences: sampled posterior thetas of shape (M, N)
    :param actions_star: 2D array of shape (n_top_actions, assortment_size)
    :param counts: 1D array of shape (n_top_actions,) = how many theta for each opt action 
    :param thetas: 1D array of the indices in [0, M-1] of the thetas associated w/ each opt action
    :return:
    """
    expected_reward_action = numba_expected_reward(pref=sampled_preferences, action=action)
    M = sampled_preferences.shape[0]
    K = action.shape[0]
    n_opt_actions = actions_star.shape[0]

    pick_given_action = np.zeros(shape=(M,))
    for i in range(M):
        sum_row = 0.
        for ix in range(K):
            sum_row += sampled_preferences[i, action[ix]]
        pick_given_action[i] =  sum_row / (1 + sum_row)
    
    probs = 0.
    theta_start = 0
    v_a = 0
    for j in range(n_opt_actions):
        p_star = counts[j] / M
        p_pick_a_star_action = 0.
        theta_indices = thetas[theta_start:theta_start+counts[j]]
        for theta_indice in theta_indices:
            p_pick_a_star_action += pick_given_action[theta_indice]
        theta_start += counts[j]
        p_pick_a_star_action /= counts[j]
        v_a += p_star * (p_pick_a_star_action - expected_reward_action) ** 2
        probs += p_star
    return v_a

In [27]:
%%timeit
ids_action_selection_numba(5, 3, delta_2, v_ids_numba, actions_set, prefs, rst, actions_st, counts_st, thetas_st)

241 µs ± 28.9 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [29]:
def information_ratio_(rho, d1, d2, g1, g2):
    return (d1 * rho + (1 - rho) * d2) ** 2 / (g1 * rho + (1 - rho) * g2)


def optimized_ratio(d1, d2, g1, g2):
    func = partial(information_ratio_, d1=d1, d2=d2, g1=g1, g2=g2)
    possible_values = [(rho, func(rho=rho)) for rho in RHO_VALUES]
    min_ = np.inf
    rho_min = -1
    for (rho_, val) in possible_values:
        if val < min_:
            rho_min = rho_
            min_ = val
    return min_, rho_min
    # solution = minimize_scalar(fun=func, bounds=(0., 1.), method='bounded')
    # return solution.fun, solution.x


def expected_reward(preferences, action):
    """
    :param preferences: shape (m, n) sampled model parameters
    :param action: indexes in [0, ..., n-1]
    :return:
    """
    filtered_item_weights = preferences[:, action].sum(1)
    return (filtered_item_weights / (1 + filtered_item_weights)).mean()


def delta_full(action, sampled_preferences, r_star):
    return r_star - expected_reward(action=action, preferences=sampled_preferences)

def g_full(action, sampled_preferences, opt_actions):
    """
    :param action:
    :param sampled_preferences: sampled posterior thetas
    :param opt_actions: dictionary {action_tuple:p_action, theta_indices}
    :return:
    """
    g_a = 0.
    probs = 0.
    M = sampled_preferences.shape[0]
    probas_given_action = sampled_preferences[:, action]
    probas_given_action = probas_given_action / (1 + np.expand_dims(probas_given_action.sum(1), axis=-1))
    no_pick_given_action = 1 - probas_given_action.sum(1)
    p_no_item_action = no_pick_given_action.mean()

    probs += p_no_item_action
    for action_star, (p_star, theta_indices) in opt_actions.items():
        p_no_item_a_star_action = np.sum([no_pick_given_action[theta_indice] for theta_indice in theta_indices]) / M
        g_a += p_no_item_a_star_action * np.log(p_no_item_a_star_action / (p_star * p_no_item_action))

    for action_ix, item_ix in enumerate(action):
        p_item_action = probas_given_action[:, action_ix].mean()
        if p_item_action:
            probs += p_item_action
            for action_star, (p_star, theta_indices) in opt_actions.items():
                p_item_a_star_action = np.sum(
                    [probas_given_action[theta_indice, action_ix] for theta_indice in theta_indices]) / M
                if p_item_a_star_action:
                    g_a += p_item_a_star_action * np.log(p_item_a_star_action / (p_star * p_item_action))
    assert probs > 0.999, f"{probs}"
    assert probs < 1.001, f"{probs}"
    return g_a

def ids_action_selection(n, k, actions_set, delta_, g_, opt_dict, sampled_prefs, r_star):
    np.random.shuffle(actions_set)
    min_information_ratio = np.inf
    ids_action = actions_set[0]
    for action1 in actions_set:
        g_a1 = g_(action1, sampled_prefs, opt_dict)
        delta_1 = delta_(action1, sampled_prefs, r_star)
        for action2 in actions_set:
            g_a2 = g_(action2, sampled_prefs, opt_dict)
            delta_2 = delta_(action2, sampled_prefs, r_star)
            if (not g_a1) or (not g_a2):
                if delta_1 < delta_2:
                    value = delta_1
                    action_picked = action1
                else:
                    value = delta_2
                    action_picked = action2
                rho = 1. if delta_1 < delta_2 else 0.
            else:
                value, rho = optimized_ratio(d1=delta_1,
                                             d2=delta_2,
                                             g1=g_a1,
                                             g2=g_a2)

                action_picked = action1 if np.random.rand() <= rho else action2
            if value < min_information_ratio:
#                 print(value)
                min_information_ratio = value
                ids_action = action_picked
#     print(ids_action)
    return ids_action

In [30]:
%%timeit
ids_action_selection(5, 3, actions_set, delta_full, g_full, opt_actions, prefs, rst)

57.2 ms ± 2.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [115]:
g_full(np.array([3, 4]), prefs, opt_actions)

0.060343818401726076

In [25]:
delta_full(action, prefs, rst)

0.37914328747832815

In [26]:
action

array([0, 1])

In [27]:
delta_full(action, prefs, rst)

0.37914328747832815

In [120]:
for action in actions_set:
    print(g_full_numba(action, prefs, actions_st, counts_st, thetas_st) - 2 * v_ids_numba(action, prefs, actions_st, counts_st, thetas_st))

0.03459037319614093
0.03566701150022636
0.04075719316445582
0.0443182500811281
0.02900209390674044
0.03923925798099936
0.021283333074070827
0.047305260793920965
0.0478405503835629
0.05150964027535662


In [29]:
opt_actions

{(3, 4): (0.2, [0, 2, 14]),
 (0, 2): (0.06666666666666667, [1]),
 (1, 3): (0.06666666666666667, [3]),
 (0, 3): (0.06666666666666667, [4]),
 (1, 4): (0.3333333333333333, [5, 7, 8, 10, 11]),
 (1, 2): (0.06666666666666667, [6]),
 (0, 1): (0.06666666666666667, [9]),
 (0, 4): (0.06666666666666667, [12]),
 (2, 4): (0.06666666666666667, [13])}

In [30]:
actions_st

array([[3, 4],
       [0, 2],
       [1, 3],
       [0, 3],
       [1, 4],
       [1, 2],
       [0, 1],
       [0, 4],
       [2, 4]])

In [31]:
counts_st

array([3, 1, 1, 1, 5, 1, 1, 1, 1])

In [32]:
thetas_st

array([ 0,  2, 14,  1,  3,  4,  5,  7,  8, 10, 11,  6,  9, 12, 13])

In [33]:
d1 = np.random.rand()
d2 = np.random.rand()
g1 = np.random.rand()
g2 = np.random.rand()

In [34]:
optimized_ratio(d1, d2, g1, g2)

(0.031639255983517856, 0.0)

In [35]:
optimized_ratio_numba(d1, d2, g1, g2)

(0.031639255983517856, 0.0)